In [136]:
import os
import pandas as pd
import numpy as np

In [137]:
#Location of folder that we want to retrieve Excel files from
path = r"REDACTED"
#Get list of files in folder
file_list = os.listdir(path)
#Remove "Ekstra" folder from list
if 'Ekstra' in file_list:
    file_list.remove('Ekstra')
os.chdir(path)

In [138]:
headers = []
#Column names that we want to retrieve from data sheet
relevant_keys = ["CVR", 
                 "Firmanavn", 
                 "Personnavn", 
                 "Postnummer", 
                 "Telefon", 
                 "Telefon 2", 
                 "Telefon 3", 
                 "Værdipapirer og kapitalandele", 
                 "Likvider", 
                 "Omsætningsaktiver", 
                 "Email", 
                 "Reklamebeskyttet"]

#List of all potential customers
emner = []

for file in file_list:
    try:
        #Storing column indices that are present in the given Excel sheet
        temp_list = []
        df = pd.read_excel(file)
        #Get a list of column names
        columns = df.columns.tolist()

        #Get relevant indices of columns in Excel sheet
        for key in relevant_keys:
            #If relevant column name is present in column names of given Excel sheet
            if key in columns:
                #Append temp_list with the column index
                temp_list.append(columns.index(key))
                continue
            temp_list.append(-1)

        #Get a list of customers in Excel sheet
        values = df.values.tolist()
        #Convert to numpy array for slicing
        values = np.array(values)
        #Empty list of same length as values, to append to missing columns
        empty_list = [""] * len(values)
        #List of customers that we want to append to final list, emner
        customers = []

        #Now we loop over the column indices
        for ind in temp_list:
            #If index is -1, we append the empty list
            if ind == -1:
                customers.append(empty_list)
                continue
            #If index is not -1, we append all values of the column index = ind
            customers.append(values[:,ind].tolist())
        #Append 'customers' list to emner
        emner.append(customers)
        print(f'Succesfully appended file: {file}')
    except:
        print(f'Could not open file: {file}')
print('Finished')

Succesfully appended file: Masterark til leads - 2019.xlsx
Succesfully appended file: Masterark til leads - 2020.xlsx
Succesfully appended file: Masterark til leads - 2021.xlsx
Finished


In [139]:
emner2 = emner.copy()
CVR = []
Firmanavn = []
Personnavn = []
Postnummer = []
Telefon = []
Telefon_2 = []
Telefon_3 = []
Værdipapirer_og_kapitalandele = []
Likvider = []
Omsætningsaktiver = []
Email = []
Reklamebeskyttet = []

for sheet in emner2:
    for sublist in range(len(sheet)):
        if sublist == 0:
            for item in sheet[sublist]:
                CVR.append(str(item).split(".")[0])
        if sublist == 1:
            for item in sheet[sublist]:
                Firmanavn.append(item)
        if sublist == 2:
            for item in sheet[sublist]:
                Personnavn.append(item)
        if sublist == 3:
            for item in sheet[sublist]:
                try:
                    Postnummer.append(int(item))
                except:
                    Postnummer.append(0)
        if sublist == 4:
            for item in sheet[sublist]:
                Telefon.append(str(item).split(".")[0])
        if sublist == 5:
            for item in sheet[sublist]:
                Telefon_2.append(str(item).split(".")[0])
        if sublist == 6:
            for item in sheet[sublist]:
                Telefon_3.append(str(item).split(".")[0])
        if sublist == 7:
            for item in sheet[sublist]:
                Værdipapirer_og_kapitalandele.append(str(item).split(".")[0])
        if sublist == 8:
            for item in sheet[sublist]:
                Likvider.append(str(item).split(".")[0])
        if sublist == 9:
            for item in sheet[sublist]:
                Omsætningsaktiver.append(str(item).split(".")[0])
        if sublist == 10:
            for item in sheet[sublist]:
                Email.append(item)
        if sublist == 11:
            for item in sheet[sublist]:
                Reklamebeskyttet.append(item)
print('Finished')

Finished


In [ ]:
df = pd.DataFrame()
df["CVR"] = CVR
df["Firmanavn"] = Firmanavn
df["Personnavn"] = Personnavn
df["Postnummer"] = Postnummer
df["Telefon"] = Telefon
df["Telefon 2"] = Telefon_2
df["Telefon 3"] = Telefon_3
df["Værdipapirer og kapitalandele"] = Værdipapirer_og_kapitalandele
df["Likvider"] = Likvider
df["Omsætningsaktiver"] = Omsætningsaktiver
df["Email"] = Email
df["Reklamebeskyttet"] = Reklamebeskyttet
length = len(df)
df = df.drop_duplicates(['CVR'], ignore_index = True)
print(f'Dropped {length-len(df)} duplicates')
length = len(df)
df = df[df.Reklamebeskyttet != 'True']
print(f'Dropped {length-len(df)} reklamebeskyttede')
del df['Reklamebeskyttet']
for column in df.columns:
    df[column] = df[column].replace(['nan'], 0)
    df[column] = df[column].fillna(0)
df = df[df.CVR != 0]
df = df[df.CVR != ""]
df = df[(df['Telefon'] != "") & (df['Telefon 2'] != "") & (df['Telefon 3'] != "")]
df = df.reset_index()
del df['index']
df

In [148]:
dføst = df.loc[df['Postnummer'] < 5000]
dføst = dføst.reset_index()
del dføst['index']
dfvest = df.loc[df['Postnummer'] >= 5000]
dfvest = dfvest.reset_index()
del dfvest['index']

filename = 'Samlet.xlsx'

writer = pd.ExcelWriter(filename)
df.to_excel(writer, index = False, header = True, sheet_name = "Samlet")
dføst.to_excel(writer, index = False, header = True, sheet_name = "Samlet-øst")
dfvest.to_excel(writer, index = False, header = True, sheet_name = "Samlet-vest")
writer.close()
print("Finished")

Finished


In [ ]:
dføst

In [ ]:
dføst = dføst[(dføst['Telefon'] != "") & (dføst['Telefon 2'] != "") & (dføst['Telefon 3'] != "")]
dføst